<a href="https://colab.research.google.com/github/nadaa-medhat1/Concrete-Crack-Detection/blob/main/notebooks/train-yolo11-object-detection-on-custom-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Train YOLO11 Object Detection on a Custom Dataset

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

YOLO11 builds on the advancements introduced in YOLOv9 and YOLOv10 earlier this year, incorporating improved architectural designs, enhanced feature extraction techniques, and optimized training methods.

YOLO11m achieves a higher mean mAP score on the COCO dataset while using 22% fewer parameters than YOLOv8m, making it computationally lighter without sacrificing performance.

YOLOv11 is available in 5 different sizes, ranging from `2.6M` to `56.9M` parameters, and capable of achieving from `39.5` to `54.7` mAP on the COCO dataset.

## Setup

### Configure API keys

To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Fri Feb 28 14:51:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLO11 via Ultralytics

In [3]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 35.7/112.6 GB disk)


## Inference with model pre-trained on COCO dataset

### CLI

**NOTE:** CLI requires no customization or Python code. You can simply run all tasks from the terminal with the yolo command.

In [4]:
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

100% 5.35M/5.35M [00:00<00:00, 90.7MB/s]
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

100% 104k/104k [00:00<00:00, 75.9MB/s]
image 1/1 /content/dog.jpeg: 640x384 2 persons, 1 car, 1 dog, 1 handbag, 65.9ms
Speed: 13.3ms preprocess, 65.9ms inference, 444.2ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


**NOTE:** Result annotated image got saved in `{HOME}/runs/detect/predict/`. Let's display it.

In [5]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/predict/dog.jpeg', width=600)

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/predict/dog.jpeg'

### SDK

**NOTE:** YOLO's Python interface allows for seamless integration into your Python projects, making it easy to load, run, and process the model's output.

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11n.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.25)[0]

**NOTE:** The obtained `result` object stores information about the location, classes, and confidence levels of the detected objects.

In [ ]:
result.boxes.xyxy

In [ ]:
result.boxes.conf

In [ ]:
result.boxes.cls

**NOTE:** YOLO11 can be easily integrated with `supervision` using the familiar `from_ultralytics` connector.

In [ ]:
import supervision as sv

detections = sv.Detections.from_ultralytics(result)

In [ ]:
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)

annotated_image = image.copy()
annotated_image = box_annotator.annotate(annotated_image, detections=detections)
annotated_image = label_annotator.annotate(annotated_image, detections=detections)

sv.plot_image(annotated_image, size=(10, 10))

## Fine-tune YOLO11 on custom dataset

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/liangdianzhong/-qvdww) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

In [6]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

rf = Roboflow(api_key="lRCaQ9UNobJwzjZovMTR")
project = rf.workspace("roboflow-universe-projects").project("construction-site-safety")
version = project.version(25)
dataset = version.download("yolov11")

/content/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Construction-Site-Safety-25 in yolov11:: 100%|██████████| 5546/5546 [00:01<00:00, 4982.32it/s]


## Custom Training

In [13]:
import os
import shutil
import json
import random
from collections import defaultdict
from roboflow import Roboflow
from ultralytics import YOLO
from google.colab import drive


# ---------------------------
# الخطوة 2: تصفية البيانات
# ---------------------------
def filter_dataset(dataset_path, max_samples=1000):
    classes_to_keep = {5, 6, 8, 9}  # Person, Safety Cone, machinery, vehicle
    filtered_path = f"{dataset_path}_filtered"

    # إنشاء هيكل المجلدات
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(filtered_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(filtered_path, split, 'labels'), exist_ok=True)

    class_instances = defaultdict(list)

    # جمع العينات لكل فئة
    for split in ['train', 'valid', 'test']:
        labels_dir = os.path.join(dataset_path, split, 'labels')
        if not os.path.exists(labels_dir):
            continue

        for filename in os.listdir(labels_dir):
            if filename.endswith('.txt'):
                file_info = {'filename': filename, 'split': split, 'classes': set()}
                with open(os.path.join(labels_dir, filename), 'r') as f:
                    for line in f:
                        class_id = int(line.split()[0])
                        if class_id in classes_to_keep:
                            file_info['classes'].add(class_id)
                            class_instances[class_id].append(file_info)

    # اختيار العينات العشوائية
    selected_files = set()
    for class_id in classes_to_keep:
        instances = class_instances[class_id]
        random.shuffle(instances)
        selected = instances[:max_samples]
        for file_info in selected:
            selected_files.add((file_info['split'], file_info['filename']))

    # نسخ الملفات المختارة
    filtered_counts = defaultdict(int)
    total_processed = 0

    for split, filename in selected_files:
        labels_dir = os.path.join(dataset_path, split, 'labels')
        images_dir = os.path.join(dataset_path, split, 'images')
        image_name = filename.replace('.txt', '.jpg')

        # معالجة الملفات
        new_annotations = []
        with open(os.path.join(labels_dir, filename), 'r') as f:
            for line in f:
                parts = line.split()
                class_id = int(parts[0])
                if class_id in classes_to_keep:
                    new_class_id = list(sorted(classes_to_keep)).index(class_id)
                    new_line = f"{new_class_id} {' '.join(parts[1:])}\n"
                    new_annotations.append(new_line)
                    filtered_counts[class_id] += 1

        # نسخ الصور والملفات
        shutil.copy(
            os.path.join(images_dir, image_name),
            os.path.join(filtered_path, split, 'images', image_name)
        )

        with open(os.path.join(filtered_path, split, 'labels', filename), 'w') as f:
            f.writelines(new_annotations)

        total_processed += 1
        if total_processed % 100 == 0:
            print(f"تم معالجة {total_processed} صورة...")

    # إنشاء ملف data.yaml
    yaml_content = f"""path: {os.path.abspath(filtered_path)}
train: {os.path.abspath(os.path.join(filtered_path, 'train', 'images'))}
val: {os.path.abspath(os.path.join(filtered_path, 'valid', 'images'))}
test: {os.path.abspath(os.path.join(filtered_path, 'test', 'images'))}

nc: 4
names:
- Person
- Safety Cone
- machinery
- vehicle"""

    with open(os.path.join(filtered_path, 'data.yaml'), 'w') as f:
        f.write(yaml_content)

    # حفظ الإحصائيات
    stats = {
        'filtered_counts': {
            'Person': filtered_counts[5],
            'Safety Cone': filtered_counts[6],
            'machinery': filtered_counts[8],
            'vehicle': filtered_counts[9]
        },
        'total_images': total_processed,
        'max_samples_per_class': max_samples,
        'dataset_path': os.path.abspath(filtered_path)
    }

    with open(os.path.join(filtered_path, 'statistics.json'), 'w') as f:
        json.dump(stats, f, indent=4)

    print("\nالنتائج النهائية:")
    for class_name, count in stats['filtered_counts'].items():
        print(f"{class_name}: {count} عينة")
    print(f"\nإجمالي الصور بعد التصفية: {total_processed}")
    print(f"\nمسار مجموعة البيانات: {os.path.abspath(filtered_path)}")

    return stats

# ---------------------------
# الخطوة 3: تدريب النموذج
# ---------------------------
#def train_model():
    #drive.mount('/content/drive')
    #DRIVE_PATH = "/content/drive/MyDrive/Construction_Models2/"
filter_dataset(dataset_path="/content/datasets/Construction-Site-Safety-25")

تم معالجة 100 صورة...
تم معالجة 200 صورة...
تم معالجة 300 صورة...
تم معالجة 400 صورة...
تم معالجة 500 صورة...
تم معالجة 600 صورة...
تم معالجة 700 صورة...
تم معالجة 800 صورة...
تم معالجة 900 صورة...
تم معالجة 1000 صورة...
تم معالجة 1100 صورة...
تم معالجة 1200 صورة...
تم معالجة 1300 صورة...
تم معالجة 1400 صورة...
تم معالجة 1500 صورة...
تم معالجة 1600 صورة...
تم معالجة 1700 صورة...

النتائج النهائية:
Person: 6862 عينة
Safety Cone: 2932 عينة
machinery: 3871 عينة
vehicle: 1392 عينة

إجمالي الصور بعد التصفية: 1768

مسار مجموعة البيانات: /content/datasets/Construction-Site-Safety-25_filtered


{'filtered_counts': {'Person': 6862,
  'Safety Cone': 2932,
  'machinery': 3871,
  'vehicle': 1392},
 'total_images': 1768,
 'max_samples_per_class': 1000,
 'dataset_path': '/content/datasets/Construction-Site-Safety-25_filtered'}

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11l.pt data=/content/datasets/Construction-Site-Safety-25_filtered/data.yaml epochs=100 imgsz=640 plots=True

/content
New https://pypi.org/project/ultralytics/8.3.81 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/content/datasets/Construction-Site-Safety-25_filtered/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

**NOTE:** The results of the completed training are saved in `{HOME}/runs/detect/train/`. Let's examine them.

In [9]:
!ls {HOME}/runs/detect/train/

args.yaml  events.out.tfevents.1740754620.4e2e3e6658d1.1991.0  weights


In [10]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train/confusion_matrix.png'

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate fine-tuned model

In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Inference with custom model

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob(f'{HOME}/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[:3]:
    display(IPyImage(filename=img, width=600))
    print("\n")

## Deploy model on Roboflow

Once you have finished training your YOLOv11 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv11 weights.

## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.